<a href="https://colab.research.google.com/github/Neerajpokala/Gemini_Quickstarts/blob/master/Video%20Analysis/Gemini_Video_Analyis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemini API: Prompting with Video

In [ ]:
!pip install -U google-generativeai

In [ ]:
import google.generativeai as genai

In [ ]:
import os
os.environ['GOOGLE_API_KEY'] = 'YOUR_GOOGLE_API_KEY'

In [ ]:
GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [ ]:
!wget https://download.blender.org/peach/bigbuckbunny_movies/BigBuckBunny_320x180.mp4

--2024-06-07 09:02:08--  https://download.blender.org/peach/bigbuckbunny_movies/BigBuckBunny_320x180.mp4
Resolving download.blender.org (download.blender.org)... 104.22.64.163, 172.67.14.163, 104.22.65.163, ...
Connecting to download.blender.org (download.blender.org)|104.22.64.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64657027 (62M) [video/mp4]
Saving to: ‘BigBuckBunny_320x180.mp4’

BigBuckBunny_320x18 100%[===================>]  61.66M  46.9MB/s    in 1.3s    

2024-06-07 09:02:09 (46.9 MB/s) - ‘BigBuckBunny_320x180.mp4’ saved [64657027/64657027]



In [ ]:
video_file_name = "BigBuckBunny_320x180.mp4"

print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/cxx4alfi81lv


## Get File

After uploading the file, you can verify the API has successfully received the files by calling `files.get`.

`files.get` lets you see the file uploaded to the File API that are associated with the Cloud project your API key belongs to. Only the `name` (and by extension, the `uri`) are unique.

In [ ]:
import time

while video_file.state.name == "PROCESSING":
    print('Waiting for video to be processed.')
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)
print(f'Video processing complete: ' + video_file.uri)

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/u9hwqc7yo4u9


## Generate Content

After the video has been uploaded, you can make `GenerateContent` requests that reference the File API URI.

In [ ]:
# Create the prompt.
prompt = "Describe this video."

# Set the model to Gemini 1.5 Flash.
model = genai.GenerativeModel(model_name="models/gemini-1.5-flash")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([prompt, video_file],
                                  request_options={"timeout": 600})
print(response.text)

Making LLM inference request...
The video starts with a scene of a peaceful forest, with a  pink cloud in the sky. Then the title card appears with the words "THE PEACH OPEN MOVIE PROJECT PRESENTS". 
The next scene shows a large, overweight bunny in a hole. The bunny yawns and stretches and then gets up to leave his hole.
There are many more scenes of the bunny in the forest, the other characters include a squirrel, a chinchilla, and a bird.
There is a lot of humorous physical comedy and some slapstick violence.
The bunny seems to be a bit of a "big" character, as he often bullies the other animals in the video. The ending shows the other animals getting their revenge against the bunny.


## Delete File

Files are automatically deleted after 2 days or you can manually delete them using `files.delete()`.

In [ ]:
genai.delete_file(video_file.name)
print(f'Deleted file {video_file.uri}')

Deleted file https://generativelanguage.googleapis.com/v1beta/files/u9hwqc7yo4u9
